<a href="https://colab.research.google.com/github/LokeshVadlamudi/EmergingTechnologiesML/blob/master/master/Assignment_4/assignment4MTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip3 uninstall keras
!pip3 install keras --upgrade

Uninstalling Keras-2.4.3:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.4.3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.4.3


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
from keras import models
from keras import layers
from keras import optimizers
import random 
from keras.preprocessing.image import ImageDataGenerator
from random import randrange
import cv2
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.layers import Activation, Flatten, Dropout, BatchNormalization

# K.tensorflow_backend._get_available_gpus()


dat = pd.read_csv('/content/drive/My Drive/MTL assignment/fgo_multiclass_labels.csv')

In [3]:

dat.head()

dat.columns

Index(['Unnamed: 0', 'full_path', 'image_name', 'name', 'white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver', 'gender_Female',
       'gender_Male', 'region_Asia', 'region_Egypt', 'region_Europe',
       'region_Middle East', 'fighting_type_magic', 'fighting_type_melee',
       'fighting_type_ranged', 'alignment_CE', 'alignment_CG', 'alignment_CN',
       'alignment_LE', 'alignment_LG', 'alignment_LN', 'alignment_NE',
       'alignment_NG', 'alignment_TN'],
      dtype='object')

In [4]:
X = dat['image_name']
y = dat[['white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver', 'gender_Female',
       'gender_Male', 'region_Asia', 'region_Egypt', 'region_Europe',
       'region_Middle East', 'fighting_type_magic', 'fighting_type_melee',
       'fighting_type_ranged', 'alignment_CE', 'alignment_CG', 'alignment_CN',
       'alignment_LE', 'alignment_LG', 'alignment_LN', 'alignment_NE',
       'alignment_NG', 'alignment_TN']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [5]:
y_train.columns


Index(['white', 'red', 'green', 'black', 'blue', 'purple', 'gold', 'silver',
       'gender_Female', 'gender_Male', 'region_Asia', 'region_Egypt',
       'region_Europe', 'region_Middle East', 'fighting_type_magic',
       'fighting_type_melee', 'fighting_type_ranged', 'alignment_CE',
       'alignment_CG', 'alignment_CN', 'alignment_LE', 'alignment_LG',
       'alignment_LN', 'alignment_NE', 'alignment_NG', 'alignment_TN'],
      dtype='object')

In [6]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

#train
colors = ['white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver']
color_train = y_train[colors]
color_nodes = color_train.shape[1]
color_train = color_train.values.tolist()

gender = ['gender_Female',
       'gender_Male']
gender_train = y_train[gender]
gender_nodes = gender_train.shape[1]
gender_train = gender_train.values.tolist()

region = ['region_Asia', 'region_Egypt', 'region_Europe',
       'region_Middle East']
region_train = y_train[region]
region_nodes = region_train.shape[1]
region_train = region_train.values.tolist()

fighting_style = ['fighting_type_magic', 'fighting_type_melee',
       'fighting_type_ranged']
fighting_style_train = y_train[fighting_style]
fighting_nodes = fighting_style_train.shape[1]
fighting_style_train = fighting_style_train.values.tolist()

alignment = ['alignment_CE', 'alignment_CG', 'alignment_CN',
       'alignment_LE', 'alignment_LG', 'alignment_LN', 'alignment_NE',
       'alignment_NG', 'alignment_TN']
alignment_train = y_train[alignment]
alignment_nodes = alignment_train.shape[1]
alignment_train = alignment_train.values.tolist()

print(color_nodes,gender_nodes,region_nodes,fighting_nodes,alignment_nodes)
#test
colors = ['white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver']
color_test = y_test[colors]
color_nodes = color_test.shape[1]
color_test = color_test.values.tolist()

gender = ['gender_Female',
       'gender_Male']
gender_test = y_test[gender]
gender_nodes = gender_test.shape[1]
gender_test = gender_test.values.tolist()

region = ['region_Asia', 'region_Egypt', 'region_Europe',
       'region_Middle East']
region_test = y_test[region]
region_nodes = region_test.shape[1]
region_test = region_test.values.tolist()

fighting_style = ['fighting_type_magic', 'fighting_type_melee',
       'fighting_type_ranged']
fighting_style_test = y_test[fighting_style]
fighting_nodes = fighting_style_test.shape[1]
fighting_style_test = fighting_style_test.values.tolist()

alignment = ['alignment_CE', 'alignment_CG', 'alignment_CN',
       'alignment_LE', 'alignment_LG', 'alignment_LN', 'alignment_NE',
       'alignment_NG', 'alignment_TN']
alignment_test = y_test[alignment]
alignment_nodes = alignment_test.shape[1]
alignment_test = alignment_test.values.tolist()

print(color_nodes,gender_nodes,region_nodes,fighting_nodes,alignment_nodes)

8 2 4 3 9
8 2 4 3 9


In [10]:
# initialize the number of epochs and batch size
EPOCHS = 50
BS = 16


def image_generator_fgo(king_of_lists, bs, mode="train", aug=None):
    # loop indefinitely
    
    while True:
        # initialize our batches of images and labels
        images = []
        #labels = []
        
        color = []
        gender = []
        region = []
        fight = []
        alignment = []
        
        # keep looping until we reach our batch size
        while len(images) < bs:
            combined_label_list = []
            random_index = randrange(len(king_of_lists[0]))
            img = image.load_img('/content/drive/My Drive/MTL assignment/images/'+king_of_lists[0][random_index],target_size=(224, 224)) #read in image
            img = image.img_to_array(img)
            #img = cv2.resize(img, (224, 224))
            #F this making my own augmentations
            #rand = random.randint(1,101)
            #if rand < 50: 
            #    img = cv2.flip( img, 0 )# horizantal flip
            #rand = random.randint(1,101)
            #img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)
            
            #create labels
            gender.append(king_of_lists[1][random_index]) # gender
            region.append(np.array(king_of_lists[2][random_index])) # region
            fight.append(np.array(king_of_lists[3][random_index])) # fighting
            alignment.append(np.array(king_of_lists[4][random_index])) # alignment
            color.append(np.array(king_of_lists[5][random_index])) # color
            
            images.append(img)
            #labels.append(gender
            
        #labels = {'gender': np.array(gender), 'region': np.array(region),
        #        'fighting_style': np.array(fight),
        #         'alignment': np.array(alignment),'color': np.array(color)}
        labels = [np.array(gender),np.array(region),np.array(fight),np.array(alignment), np.array(color)]
        #labels = [gender,region,fight,alignment,color]
        # if the data augmentation object is not None, apply it
        #labels
        if aug is not None:
            (images, labels) = next(aug.flow(np.array(images),labels, batch_size=bs))
        
        #print(labels.shape)
        # yield the batch to the calling function
        yield np.array(images),  labels 

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                         horizontal_flip=True, fill_mode="nearest")

train_lists = [X_train, gender_train, region_train, fighting_style_train, alignment_train, color_train]
test_lists = [X_test, gender_test, region_test, fighting_style_test, alignment_test, color_test]
#train_lists = [X_train, region_train, fighting_style_train, alignment_train, color_train]
#test_lists = [X_test, region_test, fighting_style_test, alignment_test, color_test]
# initialize both the training and testing image generators
trainGen = image_generator_fgo(train_lists, BS, 
                               mode="train", aug=None)
testGen = image_generator_fgo(test_lists, BS, 
                              mode="train", aug=None)

In [11]:
# Color Images, Multi-Label Targets
from keras.optimizers import SGD, Adam

loss_list = ["categorical_crossentropy",'categorical_crossentropy',
'categorical_crossentropy','categorical_crossentropy',
'binary_crossentropy']

test_metrics = {'gender': 'accuracy','region': 'accuracy',
               'fighting_style': 'accuracy','alignment': 'accuracy',
               'color': 'accuracy'}
dd = 0.0
import math
def step_decay(epoch):
   initial_lrate = 0.01
   drop = 0.5
   epochs_drop = 5.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate

def multi_model(loss_list,test_metrics,dd):
    
    base_model = VGG19(weights='imagenet', include_top=False)

    #freeze all the layers
    for layer in base_model.layers[:]:
       layer.trainable = False

    
    model_input = Input(shape=(224, 224, 3))
    x = base_model(model_input)
    x = GlobalAveragePooling2D()(x)
    
    # let's add a fully-connected layer
    x = Dense(256, activation='relu')(x)
    x = Dropout(dd)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dd)(x)
    # start passing that fully connected block output to all the 
    # different model heads
    y1 = Dense(128, activation='relu')(x)
    y1 = Dropout(dd)(y1)
    y1 = Dense(64, activation='relu')(y1)
    y1 = Dropout(dd)(y1)
    
    y2 = Dense(128, activation='relu')(x)
    y2 = Dropout(dd)(y2)
    y2 = Dense(64, activation='relu')(y2)
    y2 = Dropout(dd)(y2)
    
    y3 = Dense(128, activation='relu')(x)
    y3 = Dropout(dd)(y3)
    y3 = Dense(64, activation='relu')(y3)
    y3 = Dropout(dd)(y3)

    y4 = Dense(128, activation='relu')(x)
    y4 = Dropout(dd)(y4)
    y4 = Dense(64, activation='relu')(y4)
    y4 = Dropout(dd)(y4)
    
    y5 = Dense(128, activation='relu')(x)
    y5 = Dropout(dd)(y5)
    y5 = Dense(64, activation='relu')(y5)
    y5 = Dropout(dd)(y5)
    
    #connect all the heads to their final output layers
    y1 = Dense(gender_nodes, activation='softmax',name= 'gender')(y1)
    y2 = Dense(region_nodes, activation='softmax',name= 'region')(y2)
    y3 = Dense(fighting_nodes, activation='softmax',name= 'fighting_style')(y3)
    y4 = Dense(alignment_nodes, activation='softmax',name= 'alignment')(y4)
    y5 = Dense(color_nodes, activation='sigmoid',name= 'color')(y5)
    
    model = Model(inputs=model_input, outputs=[ y1, y2, y3, y4, y5])
    
    model.compile(loss=loss_list, optimizer=SGD(lr=0.01,momentum=0.9), metrics=test_metrics)

    return model

multi_model = multi_model(loss_list,test_metrics,dd)

multi_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg19 (Functional)              (None, None, None, 5 20024384    input_4[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 512)          0           vgg19[0][0]                      
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 256)          131328      global_average_pooling2d_1[0][0] 
_______________________________________________________________________________________

In [12]:
checkpoint = ModelCheckpoint('models/best_run5_smaller.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
lrate = LearningRateScheduler(step_decay)
callbacks_list = [checkpoint,lrate]
multi_model.fit_generator(trainGen,steps_per_epoch=len(X_train) // BS,
                    validation_data=testGen,
                    validation_steps=len(X_test) // BS,
                    epochs=EPOCHS,callbacks=callbacks_list)

Epoch 1/50
22/22 [==============================] - ETA: 0s - loss: 7.3433 - gender_loss: 1.3265 - region_loss: 1.4198 - fighting_style_loss: 1.5486 - alignment_loss: 2.3436 - color_loss: 0.7049 - gender_accuracy: 0.5426 - region_accuracy: 0.4688 - fighting_style_accuracy: 0.4943 - alignment_accuracy: 0.2273 - color_accuracy: 0.2244
Epoch 00001: val_loss improved from inf to 5.04573, saving model to models/best_run5_smaller.h5
22/22 [==============================] - 96s 4s/step - loss: 7.3433 - gender_loss: 1.3265 - region_loss: 1.4198 - fighting_style_loss: 1.5486 - alignment_loss: 2.3436 - color_loss: 0.7049 - gender_accuracy: 0.5426 - region_accuracy: 0.4688 - fighting_style_accuracy: 0.4943 - alignment_accuracy: 0.2273 - color_accuracy: 0.2244 - val_loss: 5.0457 - val_gender_loss: 0.8091 - val_region_loss: 0.8701 - val_fighting_style_loss: 0.8211 - val_alignment_loss: 1.9275 - val_color_loss: 0.6180 - val_gender_accuracy: 0.5938 - val_region_accuracy: 0.7188 - val_fighting_style_a